In [0]:
import pandas as pd
import numpy as np 
from functools import reduce 
import os
from datetime import datetime

import matplotlib as mp
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None

import seaborn as sns
sns.set(style="whitegrid")
sns.set(font_scale=2)

In [0]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [0]:
from google.cloud import bigquery
client = bigquery.Client('thesisprojects2019')

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Arrival Time - Profile Graphs

In [0]:
#import statistic table from bigquery
query = """ SELECT a.eventid ,b.event, gps_1st_min5_instadium as arrival_time, start_time, end_time, 
             COUNT(distinct a.user_id) as count_spect_instadium,
             ROUND((COUNT(distinct a.user_id)/b.count_spectators)*100,2) as perc_spect_instadium
FROM  `thesisprojects2019.audeMS.step9_temporal_analysis_arrival` as a, 
            `thesisprojects2019.audeMS.EDA_spectators`as b
WHERE a.eventid = b.eventid 
GROUP BY  a.eventid, b.event, b.count_spectators, a.gps_1st_min5_instadium, start_time, end_time 
ORDER BY eventid, gps_1st_min5_instadium"""
data = client.query(query).to_dataframe()

In [0]:
data['a_time'] = data['arrival_time'].dt.time

In [0]:
data.head(2)

,eventid,event,arrival_time,start_time,end_time,count_spect_instadium,perc_spect_instadium,a_time
0,1_2017-11-04,Barbarians New Zealand,2017-11-04 11:00:00+00:00,15:00:00,16:45:00,3,0.27,11:00:00
1,1_2017-11-04,Barbarians New Zealand,2017-11-04 11:05:00+00:00,15:00:00,16:45:00,3,0.27,11:05:00


In [0]:
data.dtypes

eventid                               object
event                                 object
arrival_time             datetime64[ns, UTC]
start_time                            object
end_time                              object
count_spect_instadium                  int64
perc_spect_instadium                 float64
a_time                                object
dtype: object

In [0]:
# Create time range
date_rng = pd.date_range(start='09:00', periods=200, freq='20T')
x_list = date_rng.time

In [0]:
#Make a list of all single eventid
event_list = []
for i in data['event']:
    if i not in event_list:
        event_list.append(i)

In [0]:
## Define a function to create the plot for each single event!
def Plot_instadium(eventname):
    
    # select the event date
    data_test = data[data['event'] == eventname]
    data_event = data_test.reset_index()
    # Add the cumulative perc_gpspt 
 # data_event['cum_perc'] = 100 - data_event['perc_1stgps_outstadium'].expanding().apply(lambda r: reduce(lambda prev, value: prev + value, r))

    # select the event name
    event_name = data_event['event'][1]
    start_time = data_event['start_time'][0] 
    end_time = data_event['end_time'][0] 
    
    # define the x,y value
    x = data_event['a_time'] # here i made a change
  # y = data_event['cum_perc']
    y = data_event['perc_spect_instadium']

    # make the figure
    fig, ax = plt.subplots(figsize=(10,4),dpi=120)
  # fig, ax = plt.subplots(figsize=(10,4))
    
    # set the x_axis, x,y,title label
    #ax.set_xlim(left=data_event['time_intervall'].min(), right=data_event['time_intervall'].max())
    plt.suptitle('Spectators Arrival in Stadium - 1st GPS point', fontsize=14, horizontalalignment = 'center')
    ax.set_title((event_name),fontsize='12')
    ax.set_xlabel('Time',horizontalalignment='center', fontsize='12')
    ax.set_ylabel('% of Total Spectators',horizontalalignment='center', fontsize='12')
  # ax.set_yticks(np.arange(0,110,10))
    ax.set_yticks(np.arange(0,13,2))
    ax.set_ylim(top=12, bottom=0)
    ax.set_xticks(x_list)
    ax.tick_params(axis='both', which='major', pad=6)
        
    # draw the plot
    ax.plot(x,y)
    ax.axvline(start_time,ls='--',lw=0.9,color='g',label=('Event Start ' + str(start_time)))
    ax.axvline(end_time,ls='--',lw=0.9,color='r',label=('Event End ' + str(end_time)))
    plt.gcf().autofmt_xdate()
    plt.yticks(horizontalalignment='center', fontsize='12')
    plt.xticks(rotation=0,horizontalalignment='center', fontsize='9')
    plt.legend(loc=0,fontsize='10' )
    
   # plt.savefig('drive/My Drive/Dan - Meetings/Thesis Maps & Graphs/Arrival_Time/Arrival_Profile/graph_arrival_spect_'+str(event_name)+'.png')
    plt.close('all')
    # return the figure
    return fig, ax

In [0]:
for events in event_list:
    plot = Plot_instadium(events)  

In [0]:
import os
from google.colab import files

folderpath = '/content/drive/My Drive/Dan - Meetings/Thesis Maps & Graphs/Arrival_Time/Arrival_Profile'

for filename in os.listdir(folderpath):
 # print(filename)
  files.download(folderpath+'/'+str(filename))

### Departure Time - Profile Graphs

In [0]:
#import statistic table from bigquery
query = """ SELECT a.eventid ,b.event, gps_1st_min5_outstadium as departure_time, start_time, end_time, 
             COUNT(distinct a.user_id) as count_spect_outstadium,
             ROUND((COUNT(distinct a.user_id)/b.count_spectators)*100,2) as perc_spect_outstadium
FROM  `thesisprojects2019.audeMS.step9_temporal_analysis_departure` as a, 
            `thesisprojects2019.audeMS.EDA_spectators`as b
WHERE a.eventid = b.eventid 
GROUP BY  a.eventid, b.event, b.count_spectators, a.gps_1st_min5_outstadium, start_time, end_time 
ORDER BY eventid, gps_1st_min5_outstadium"""
data = client.query(query).to_dataframe()

In [0]:
data['d_time'] = data['departure_time'].dt.time

In [0]:
data.head(2)

,eventid,event,departure_time,start_time,end_time,count_spect_outstadium,perc_spect_outstadium,d_time
0,1_2017-11-04,Barbarians New Zealand,NaT,15:00:00,16:45:00,3,0.27,NaT
1,1_2017-11-04,Barbarians New Zealand,2017-11-04 15:10:00+00:00,15:00:00,16:45:00,2,0.18,15:10:00


In [0]:
test_data = data[data.departure_time.notnull()] 

In [0]:
test_data.set_index('departure_time', inplace=True)
test_data.index = pd.to_datetime(test_data.index)
test_data.head(2)

,eventid,event,start_time,end_time,count_spect_outstadium,perc_spect_outstadium,d_time
departure_time,,,,,,,
2017-11-04 15:10:00+00:00,1_2017-11-04,Barbarians New Zealand,15:00:00,16:45:00,2,0.18,15:10:00
2017-11-04 15:15:00+00:00,1_2017-11-04,Barbarians New Zealand,15:00:00,16:45:00,3,0.27,15:15:00


In [0]:
timestampe_search = '00:00:00'
test_new = test_data.loc[test_data.index.strftime('%H:%M:%S') != timestampe_search]
clean_data = test_new.reset_index()

In [0]:
clean_data.head(2)

,departure_time,eventid,event,start_time,end_time,count_spect_outstadium,perc_spect_outstadium,d_time
0,2017-11-04 15:10:00+00:00,1_2017-11-04,Barbarians New Zealand,15:00:00,16:45:00,2,0.18,15:10:00
1,2017-11-04 15:15:00+00:00,1_2017-11-04,Barbarians New Zealand,15:00:00,16:45:00,3,0.27,15:15:00


In [0]:
## Define a function to create the plot for each single event!
def Plot_outstadium(eventname):
    
    # select the event date
    data_test = clean_data[clean_data['event'] == eventname]
    data_event = data_test.reset_index()
    # Add the cumulative perc_gpspt 
 # data_event['cum_perc'] = 100 - data_event['perc_1stgps_outstadium'].expanding().apply(lambda r: reduce(lambda prev, value: prev + value, r))

    # select the event name
    event_name = data_event['event'][1]
    start_time = data_event['start_time'][0] 
    end_time = data_event['end_time'][0] 
    
    # define the x,y value
    x = data_event['d_time'] # here i made a change
  # y = data_event['cum_perc']
    y = data_event['perc_spect_outstadium']

    # make the figure
    fig, ax = plt.subplots(figsize=(10,4),dpi=120)
  #  fig, ax = plt.subplots(figsize=(12,4))
    
    # set the x_axis, x,y,title label
  #  ax.set_xlim(left=data_event['time_intervall'].min(), right=data_event['time_intervall'].max())
    plt.suptitle('Spectators Departure from Stadium - 1st GPS point out', fontsize=14, horizontalalignment = 'center')
    ax.set_title((event_name),fontsize='12')
    ax.set_xlabel('Time',horizontalalignment='center', fontsize='12')
    ax.set_ylabel('% of Total Spectators',horizontalalignment='center', fontsize='12')
  # ax.set_yticks(np.arange(0,110,10))
    ax.set_yticks(np.arange(0,25,3))
    ax.set_ylim(top=25, bottom=0)
    ax.set_xticks(x_list)
    ax.tick_params(axis='both', which='major', pad=6)
        
    # draw the plot
    ax.plot(x,y)
    ax.axvline(start_time,ls='--',lw=0.9,color='g',label=('Event Start ' + str(start_time)))
    ax.axvline(end_time,ls='--',lw=0.9,color='r',label=('Event End ' + str(end_time)))
    plt.gcf().autofmt_xdate()
    plt.yticks(horizontalalignment='center', fontsize='12')
    plt.xticks(rotation=0,horizontalalignment='center', fontsize='8')
    plt.legend(loc=0,fontsize='10' )
    
    plt.savefig('drive/My Drive/Dan - Meetings/Thesis Maps & Graphs/Departure_Time/1st_GPS_outstadium/Departure_Profile/graph_departure_spect_'+str(event_name)+'.png')
    plt.close('all')
    # return the figure
    return fig, ax

In [0]:
#Plot_outstadium('Watford Leicester') # Set ylim to 24!

# Plot_outstadium('THFC Southampton')# Set width = 12
# Plot_outstadium('Barbarians New Zealand') # Set width = 12
# Plot_outstadium('England Argentina') # Set width = 12
# Plot_outstadium('England Samoa') # Set width = 12
# Plot_outstadium('West Ham Chelsea') # Set width = 12
# Plot_outstadium('NFL Game 1') # Set width = 12
# Plot_outstadium('NFL Game 2') # Set width = 12
# Plot_outstadium('THFC Bournemouth ') # Set width = 12
# Plot_outstadium('THFC Liverpool') # Set width = 12
# Plot_outstadium('THFC CrystalPalace') # Set width = 12
# Plot_outstadium('THFC WBA') # Set width = 12
# Plot_outstadium('THFC Stoke') # Set width = 12

(<Figure size 1200x480 with 1 Axes>,
 <matplotlib.axes._subplots.AxesSubplot at 0x7fb4b95386a0>)

In [0]:
for events in event_list:
    plot = Plot_outstadium(events)  

In [0]:
import os
from google.colab import files

folderpath = '/content/drive/My Drive/Dan - Meetings/Thesis Maps & Graphs/Departure_Time/1st_GPS_outstadium/Departure_Profile'

for filename in os.listdir(folderpath):
 # print(filename)
  files.download(folderpath+'/'+str(filename))

### Departure Time - Cummulative Percentage Profile Graphs

In [0]:
## Define a function to create the plot for each single event!
def Plot_outstadium(eventname):
    
    # select the event date
    data_test = clean_data[clean_data['event'] == eventname]
    data_event = data_test.reset_index()
    # Add the cumulative perc_gpspt 
    data_event['cum_perc'] = 100 - data_event['perc_spect_outstadium'].expanding().apply(lambda r: reduce(lambda prev, value: prev + value, r))

    # select the event name
    event_name = data_event['event'][1]
    start_time = data_event['start_time'][0] 
    end_time = data_event['end_time'][0] 
    
    # define the x,y value
    x = data_event['d_time'] # here i made a change
    y = data_event['cum_perc']

    # make the figure
    fig, ax = plt.subplots(figsize=(13,4),dpi=120)
   # fig, ax = plt.subplots(figsize=(10,4))
    
    # set the x_axis, x,y,title label
    #ax.set_xlim(left=data_event['time_intervall'].min(), right=data_event['time_intervall'].max())
    plt.suptitle('Spectators Departure from Stadium - 1st GPS point out', fontsize=14, horizontalalignment = 'center')
    ax.set_title((event_name),fontsize='12')
    ax.set_xlabel('Time',horizontalalignment='center', fontsize='12')
    ax.set_ylabel('Cumulative % of Total Spectators',horizontalalignment='center', fontsize='12')
    ax.set_yticks(np.arange(0,110,10))
  #  ax.set_ylim(top=17, bottom=0)
    ax.set_xticks(x_list)
    ax.tick_params(axis='both', which='major', pad=6)
        
    # draw the plot
    ax.plot(x,y)
    ax.axvline(start_time,ls='--',lw=0.9,color='g',label=('Event Start ' + str(start_time)))
    ax.axvline(end_time,ls='--',lw=0.9,color='r',label=('Event End ' + str(end_time)))
    plt.gcf().autofmt_xdate()
    plt.yticks(horizontalalignment='center', fontsize='12')
    plt.xticks(rotation=0,horizontalalignment='center', fontsize='7')
    plt.legend(loc=0,fontsize='10' )
    
    plt.savefig('drive/My Drive/Dan - Meetings/Thesis Maps & Graphs/Departure_Time/1st_GPS_outstadium/Departure_Cum/cum_departure_spect_'+str(event_name)+'.png')
    plt.close('all')
    # return the figure
    return fig, ax

In [0]:
# Plot_outstadium('Watford Leicester') # Set ylim to 24!

# Plot_outstadium('THFC Southampton')# Set width = 12
# Plot_outstadium('Barbarians New Zealand') # Set width = 12
# Plot_outstadium('England Argentina') # Set width = 12
# Plot_outstadium('England Samoa') # Set width = 12
# Plot_outstadium('West Ham Chelsea') # Set width = 12
# Plot_outstadium('NFL Game 1') # Set width = 12
# Plot_outstadium('NFL Game 2') # Set width = 12
# Plot_outstadium('THFC Bournemouth ') # Set width = 12
# Plot_outstadium('THFC Liverpool') # Set width = 12
# Plot_outstadium('THFC CrystalPalace') # Set width = 12
# Plot_outstadium('THFC WBA') # Set width = 12
# Plot_outstadium('THFC Stoke') # Set width = 12

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  import sys


(<Figure size 1560x480 with 1 Axes>,
 <matplotlib.axes._subplots.AxesSubplot at 0x7fb4b6e4e390>)

In [0]:
for events in event_list:
    plot = Plot_outstadium(events)  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  import sys


In [0]:
import os
from google.colab import files

folderpath = '/content/drive/My Drive/Dan - Meetings/Thesis Maps & Graphs/Departure_Time/1st_GPS_outstadium/Departure_Cum'

for filename in os.listdir(folderpath):
 # print(filename)
  files.download(folderpath+'/'+str(filename))